<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Algorithmic Trading

**Chapter 11 &mdash; Automating Trading Systems**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'

## Capital Management 

### Deriving the Kelly Criterion in a Binomial Setting

In [ ]:
import numpy as np

In [ ]:
np.random.seed(1000)

In [ ]:
p = 0.55

In [ ]:
f = p - (1 - p)

In [ ]:
f

In [ ]:
I = 50  # number of series

In [ ]:
n = 100  # number of trials per series

In [ ]:
def run_simulation(f):
    c = np.zeros((n, I))
    c[0] = 100
    for i in range(I):
        for t in range(1, n):
            o = np.random.binomial(1, p)
            if o > 0:
                c[t, i] = (1 + f) * c[t - 1, i]
            else:
                c[t, i] = (1 - f) * c[t - 1, i]
    return c

In [ ]:
c_1 = run_simulation(f)

In [ ]:
c_1

In [ ]:
plt.plot(c_1, 'b', lw=0.5)
plt.plot(c_1.mean(axis=1), 'r', lw=2.5)
# plt.savefig('../../images/ch11/kelly_01.png');

In [ ]:
c_2 = run_simulation(0.05)

In [ ]:
c_3 = run_simulation(0.25)

In [ ]:
c_4 = run_simulation(0.5)

In [ ]:
plt.plot(c_1.mean(axis=1), 'r', label='$f^*=0.1$')
plt.plot(c_2.mean(axis=1), 'b', label='$f=0.05$')
plt.plot(c_3.mean(axis=1), 'y', label='$f=0.25$')
plt.plot(c_4.mean(axis=1), 'm', label='$f=0.5$')
plt.legend(loc=0)
# plt.savefig('../../images/ch11/kelly_02.png');

### Deriving the Kelly Criterion for Stocks and Indices

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
h5 = pd.HDFStore('../data/equities.h5', 'r')

In [ ]:
data = pd.DataFrame(h5['data']['^GSPC'])

In [ ]:
data = data[(data.index > '2010-1-1') & (data.index <= '2017-1-24')]

In [ ]:
data.columns = ['prices']

In [ ]:
data['returns'] = np.log(data / data.shift(1))

In [ ]:
data.fillna(0, inplace=True)

In [ ]:
mu = data.returns.mean() * 252

In [ ]:
mu

In [ ]:
sigma = data.returns.std() * 252 ** 0.5

In [ ]:
sigma

In [ ]:
r = 0.0

In [ ]:
f = (mu - r) / sigma ** 2 # * 0.5

In [ ]:
f 

In [ ]:
from math import exp

In [ ]:
data['equity'] = 1
data['capital'] = data['equity'] * f
for i in range(1, len(data)):
    data.ix[i, 'capital'] = data['capital'].ix[i - 1] * \
                               exp(data['returns'].ix[i])
    data.ix[i, 'equity'] = data['capital'].ix[i] - \
                             data['capital'].ix[i - 1] + \
                             data['equity'].ix[i - 1]
    data.ix[i, 'capital'] = data['equity'].ix[i] * f

In [ ]:
data.head(10)

In [ ]:
ax = data['returns'].cumsum().apply(np.exp).plot(legend=True)
data['equity'].plot(ax=ax, legend=True)
# plt.savefig('../../images/ch11/kelly_03.png');
# half Kelly
# plt.savefig('../../images/ch11/kelly_04.png');

## Risk Management

### Strategy Characteristics

In [ ]:
import configparser

In [ ]:
config = configparser.ConfigParser()

In [ ]:
config.read('../pyalgo.cfg')

In [ ]:
import v20

In [ ]:
import pandas as pd

In [ ]:
import datetime as dt

In [ ]:
ctx = v20.Context(
            'api-fxpractice.oanda.com',
            443,
            True,
            application='sample_code',
            token=config['oanda_v20']['access_token'],
            datetime_format='RFC3339'
)

In [ ]:
suffix = '.000000000Z'

In [ ]:
fromTime = dt.datetime(2017, 2, 1, 8, 0, 0).isoformat('T') + suffix

In [ ]:
fromTime

In [ ]:
toTime = dt.datetime(2017, 2, 1, 21, 0, 0).isoformat('T') + suffix

In [ ]:
%%time
res = ctx.instrument.candles(instrument='DE30_EUR',
                             fromTime=fromTime,
                             toTime=toTime,
                             granularity='S10',
                             price='A')

In [ ]:
raw = res.get('candles')

In [ ]:
raw = [cs.dict() for cs in raw]

In [ ]:
for cs in raw:
    cs.update(cs['ask'])
    del cs['ask']

In [ ]:
raw[0]

In [ ]:
data = pd.DataFrame(raw)

In [ ]:
data['time']= pd.to_datetime(data['time'])

In [ ]:
data = data.set_index('time')

In [ ]:
data[['c', 'l', 'h', 'o']] = data[
                ['c', 'l', 'h', 'o']].astype('float64')

In [ ]:
data.info()

In [ ]:
data.tail()

In [ ]:
import numpy as np

In [ ]:
data['returns'] = np.log(data['c'] / data['c'].shift(1))

In [ ]:
data['SMA1'] = data['c'].rolling(5).mean()

In [ ]:
data['SMA2'] = data['c'].rolling(10).mean()

In [ ]:
data['position'] = np.where(data['SMA1'] > data['SMA2'], 1, -1)

In [ ]:
data['strategy'] = data['position'].shift(1) * data['returns']

In [ ]:
data.dropna(inplace=True)

In [ ]:
data[['returns', 'strategy']].cumsum().apply(np.exp).plot(figsize=(10, 6))
# plt.savefig('../../images/ch11/strategy_01.png')

In [ ]:
mean = data[['returns', 'strategy']].mean() * 6 * 60 * 24 * 252
mean

In [ ]:
var = data[['returns', 'strategy']].var() * 6 * 60 * 24 * 252
var

In [ ]:
var ** 0.5

In [ ]:
mean / var

In [ ]:
mean / var * 0.5

### Leverage

In [ ]:
to_plot = ['returns', 'strategy']

In [ ]:
for l in [10, 20, 30, 40, 50, 100]:
    label = 'lstrategy_%d' % l
    data[label] = data['strategy'] * l
    to_plot.append(label)

In [ ]:
data[to_plot].cumsum().apply(np.exp).plot(figsize=(10, 6))
# plt.savefig('../../images/ch11/strategy_02.png')

In [ ]:
data.position.value_counts()

In [ ]:
sum(abs(data.position.diff()) > 1.5)

### Resulting Risk Statistics

In [ ]:
risk = pd.DataFrame(data['lstrategy_30'])

In [ ]:
risk['equity'] = risk['lstrategy_30'].cumsum().apply(np.exp) * 5000

In [ ]:
risk['cummax'] = risk['equity'].cummax()

In [ ]:
risk[['equity', 'cummax']].plot(figsize=(10, 6))
# plt.savefig('../../images/ch11/strategy_03.png')

In [ ]:
risk['drawdown'] = risk['cummax'] - risk['equity']

In [ ]:
risk['drawdown'].max()

In [ ]:
temp = risk['drawdown'][risk['drawdown'] == 0]

In [ ]:
periods = (temp.index[1:].to_pydatetime() -
           temp.index[:-1].to_pydatetime())

In [ ]:
t = periods.max()

In [ ]:
t

In [ ]:
t.seconds / 60 / 60

### Value-at-Risk

In [ ]:
import scipy.stats as scs

In [ ]:
percs = [0.01, 0.1, 1., 2.5, 5.0, 10.0]

In [ ]:
VaR = scs.scoreatpercentile(risk['equity'] - 5000, percs)

In [ ]:
VaR.round(2)

In [ ]:
def print_var():
    print('%16s %16s' % ('Confidence Level', 'Value-at-Risk'))
    print(33 * '-')
    for pair in zip(percs, VaR):
        print('%16.2f %16.3f' % (100 - pair[0], -pair[1]))

In [ ]:
print_var()

In [ ]:
risk['returns'] = np.log(risk['equity'] / risk['equity'].shift(1))

In [ ]:
risk['returns'].hist(bins=35)
# plt.savefig('../../images/ch11/strategy_04.png')

In [ ]:
VaR = scs.scoreatpercentile(5000 * risk['returns'], percs)

In [ ]:
print_var()

In [ ]:
risk_resam = risk.resample('5min').last()

In [ ]:
risk_resam['returns'] = np.log(risk_resam['equity'] /
                               risk_resam['equity'].shift(1))

In [ ]:
(5000 * risk_resam['returns']).hist(bins=35)
# plt.savefig('../../images/ch11/strategy_05.png')

In [ ]:
VaR = scs.scoreatpercentile(5000 * risk_resam['returns'], percs)

In [ ]:
print_var()

### Tail Risk

In [ ]:
h5 = pd.HDFStore('../data/equities.h5', 'r')

In [ ]:
sp500 = pd.DataFrame(h5['data']['^GSPC'])

In [ ]:
sp500.columns = ['prices']

In [ ]:
sp500.info()

In [ ]:
sp500['returns'] = np.log(sp500 / sp500.shift(1))

In [ ]:
sp500.returns.hist(bins=55)

In [ ]:
sp500.returns.min()

In [ ]:
sp500.returns.argmin()

In [ ]:
risk['lstrategy_30'].min()

In [ ]:
risk['lstrategy_30'].argmin()

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

**Python Quant Platform** |
<a href="http://quant-platform.com">http://quant-platform.com</a>

**Python for Finance** |
<a href="http://python-for-finance.com" target="_blank">Python for Finance @ O'Reilly</a>

**Derivatives Analytics with Python** |
<a href="http://derivatives-analytics-with-python.com" target="_blank">Derivatives Analytics @ Wiley Finance</a>

**Listed Volatility and Variance Derivatives** |
<a href="http://lvvd.tpq.io" target="_blank">Listed VV Derivatives @ Wiley Finance</a>

**Python Training** |
<a href="http://training.tpq.io" target="_blank">Python for Finance University Certificate</a>